In [1]:
import torch
print(torch.__version__)

In [5]:
import torchvision
import torch
import os
import time

if os.path.exists("models"):
    pass
else:
    os.mkdir("models")

# --------------定义训练的设备-----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data = torchvision.datasets.CIFAR10(
    root = "./datas",
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
test_data = torchvision.datasets.CIFAR10(
    root = "./datas",
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
train_data_size = len(train_data)

test_data_size = len(test_data)

train_dataloader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
)

teat_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
) 

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(32,32,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32,64,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(64*4*4,64),
            torch.nn.Linear(64,10)
        )
        
    def forward(self,x):
        x = self.model(x)
        return x

#--------------------------- 修改网络模型-----------------------------
model = Model()
model.to(device)

input = torch.ones((64,3,32,32))
output = model(input)
print(input.shape)
print(output.shape) 

#--------------------------- 修改损失函数-----------------------------
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

total_train_step = 0

total_test_step = 0 

epoch = 10

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./project_logs")

for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    start_time = time.time()
    model.train()
    for data in train_dataloader:
        #--------------------------- 修改网络数据-----------------------------
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step+=1
        if total_train_step % 100 == 0:
            writer.add_scalar("train_loss",loss.item(),total_train_step)
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))
    end_time = time.time()
    print("第{}轮训练时间：".format(i+1),end_time-start_time)
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in teat_dataloader:
            #--------------------------- 修改网络数据-----------------------------
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            
            
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    writer.add_scalar("test_total_loss",total_test_loss,i+1)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,i+1)
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))
    
    torch.save(model,"./models/model_{}.path".format(i+1))
    print("模型{}已保存".format(i+1))
    
writer.close()

In [3]:
import torch
print(torch.__version__)

In [4]:
print(torch.cuda.is_available())

In [16]:
import torchvision
import torch
import os
import time

if os.path.exists("models"):
    pass
else:
    os.mkdir("models")

# --------------定义训练的设备-----------------
device = torch.device("cuda")

train_data = torchvision.datasets.CIFAR10(
    root = "./datas",
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
test_data = torchvision.datasets.CIFAR10(
    root = "./datas",
    train = False,
    transform = torchvision.transforms.ToTensor(),
    download = True
)
train_data_size = len(train_data)

test_data_size = len(test_data)

train_dataloader = torch.utils.data.DataLoader(
    train_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
)

teat_dataloader = torch.utils.data.DataLoader(
    test_data,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
) 

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,stride=1,padding=2),
            torch.nn.MaxPool2d(kernel_size=2),
            torch.nn.Conv2d(32,32,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Conv2d(32,64,5,1,2),
            torch.nn.MaxPool2d(2),
            torch.nn.Flatten(),
            torch.nn.Linear(64*4*4,64),
            torch.nn.Linear(64,10)
        )
        
    def forward(self,x):
        x = self.model(x)
        return x

#--------------------------- 修改网络模型-----------------------------
model = Model()
model = model.to(device)

input = torch.ones((64,3,32,32))
output = model(input.to(device))
print(input.shape)
print(output.shape) 

#--------------------------- 修改损失函数-----------------------------
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

total_train_step = 0

total_test_step = 0 

epoch = 10

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("./project_logs")

for i in range(epoch):
    print("-------------第{}轮训练开始------------".format(i+1))
    # 训练步骤开始
    start_time = time.time()
    model.train()
    for data in train_dataloader:
        #--------------------------- 修改网络数据-----------------------------
        imgs,targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = model(imgs)
        loss = loss_fn(outputs,targets)
        
        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_train_step+=1
        if total_train_step % 100 == 0:
            writer.add_scalar("train_loss",loss.item(),total_train_step)
            print("训练次数：{},loss:{}".format(total_train_step,loss.item()))
    end_time = time.time()
    print("第{}轮训练时间：".format(i+1),end_time-start_time)
    # 测试步骤
    model.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in teat_dataloader:
            #--------------------------- 修改网络数据-----------------------------
            imgs,targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            
            outputs = model(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy+=accuracy
    writer.add_scalar("test_total_loss",total_test_loss,i+1)
    writer.add_scalar("test_accuracy",total_accuracy/test_data_size,i+1)
    print("整体测试集上的loss:{}".format(total_test_loss))
    print("整体测试集上的正确率:{}".format(total_accuracy/test_data_size))
    
    torch.save(model,"./models/model_{}.path".format(i+1))
    print("模型{}已保存".format(i+1))
    
writer.close()

In [17]:
!nvidia-smi